In [1]:
"""
Shot and decomposed prompting: decompose into identify then remove

Identify: 3-shot
Remove: 4-shot
"""

'\nShot and decomposed prompting: decompose into identify then remove\n\nIdentify: 3-shot\nRemove: 4-shot\n'

In [2]:
import sys
sys.path.append('../../')
from run_process import get_unused_data
sys.path.append('../../../')
from utils import print_check_gpt_results, stats_results_unused

In [4]:
# define variables
NUM_FILES = 209
SAMPLES_FOLDER_NAME = '../random_samples_functions'
GPT_SAVED_FILE_NAME = 'unused_functions_gpt'

In [5]:
# read in files from folder random_cells
random_cells = []

for i in range(NUM_FILES):
    file_name = f'{SAMPLES_FOLDER_NAME}/{i}.py'
    with open(file_name, 'r') as f:
        random_cells.append(f.read())

In [6]:
task1_prompt = "A function/method is unused if it is defined using 'def' but is not referenced after being defined. Identify unused functions/methods in the code delimited by triple backticks. Output this as a list of function names."

task1_ex1_input = """```python
c = 5

def calc():
    a = 5
    b = 4
    return a + b
```"""

task1_ex1_output = "['calc']"

task1_ex2_input = """```python
def multiply(a, b):
    return a * b

c = 5

def special():
    a = 5
    b = 4
    return multiply(a, b)
```"""

task1_ex2_output = "['special']"

task1_ex3_input = """```python
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def update_name(self, name):
        self.name = name
```"""

task1_ex3_output = "['update_name']"

task1_ex4_input = """```python
def seven():
    return 7
```"""

task1_ex4_output = """['seven']"""

In [7]:
# # identify unused functions using vulture (GPT)
# import openai
# openai.api_key = my_key

# # GPT
# def identify_unused(cell_src):
#     while True:
#         try:
#             completion = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo",
#                 temperature=0,
#                 messages = [
#                 {"role": "user", "content": task1_prompt},
#                 {"role": "user", "content": task1_ex1_input},
#                 {"role": "assistant", "content": task1_ex1_output},
#                 {"role": "user", "content": task1_ex2_input},
#                 {"role": "assistant", "content": task1_ex2_output},
#                 {"role": "user", "content": task1_ex3_input},
#                 {"role": "assistant", "content": task1_ex3_output},
#                 {"role": "user", "content": task1_ex4_input},
#                 {"role": "assistant", "content": task1_ex4_output},
#                 {"role": "user", "content": f"```python\n{cell_src}\n```"}
#             ]
#             )
#         except Exception as e:
#             if 'maximum context length' in str(e):
#                 print('...Error.. too long...' + str(e))
#                 return 'length', None
#             else:
#                 print('...Error.. trying again...' + str(e))
#         else:
#             break
#     return completion.choices[0].finish_reason, completion.choices[0].message["content"]

# gpt_results = []
# for i, cell_src in enumerate(random_cells):
#     print(f'Processing file {i}')
#     finish_reason, result = identify_unused(cell_src)
#     print(f'File {i} - {finish_reason}')
#     gpt_results.append({'reason': finish_reason, 'result': result})

# # save the results to a file
# with open(GPT_SAVED_FILE_NAME, 'w') as f:
#     f.write(str(gpt_results))

In [8]:
# read in gpt result from file
with open(GPT_SAVED_FILE_NAME, 'r') as f:
    gpt_results = eval(f.read())

In [9]:
# save the results to a variable
gpt_unused_names = []
for var in gpt_results:
    if var['reason'] == 'stop':
        try:
            res = eval(var['result'])
        except:
            gpt_unused_names.append(None)
        else:
            if res == []:
                gpt_unused_names.append(None)
            else:
                gpt_unused_names.append(res)
    else:
        gpt_unused_names.append(None)

In [10]:
task2_prompt = "Remove the functions specified from the code snippet enclosed by triple backticks. Do not add, modify, or remove anything else. Output the updated code with the specified functions removed."

task2_ex1_input = """Code:
```python
c = 5

def random_adder():
    a = 2
    b = 3
    return a + b + c
```

Functions to remove:
['random_adder']"""

task2_ex1_output = """```python
c = 5
```"""

task2_ex2_input = """Code:
```python
par_val = 23
def multiply(a, b):
    return a * b
new_val = 1
```

Functions to remove:
['multiply']"""

task2_ex2_output = """```python
par_val = 23
new_val = 1
```"""

task2_ex3_input = """Code:
```python
def estimate(earnings):
    return 0.1 * earnings
```

Functions to remove:
['earnings']"""

task2_ex3_output = """```python
```"""

task2_ex4_input = """Code:
```python
def seven():
    return 7

def validate(val1, val2):
    assert val1 and val2
    val3 = seven()
    return val1 * val2 - val3
```

Functions to remove:
['seven', 'validate']"""

task2_ex4_output = """```python
```"""

In [11]:
# Estimate cost
import sys
sys.path.append("../../..")
import utils

def estimate_tokens():
    in_tok = ''
    out_tok = ''
    for i, cell_src in enumerate(random_cells):
        # estimate prompt
        in_tok += task1_prompt + task1_ex1_input + task1_ex1_output + task1_ex2_input + task1_ex2_output + task1_ex3_input + task1_ex3_output + task1_ex4_input + task1_ex4_output
        in_tok += f"```python\n{cell_src}\n```"
        in_tok += task2_prompt + task2_ex1_input + task2_ex1_output + task2_ex2_input + task2_ex2_output + task2_ex3_input + task2_ex3_output + task2_ex4_input + task2_ex4_output
        in_tok += f"Code:\n```python\n{cell_src}\n```\n\nFunctions to remove:\n['test1', 'test2']"
        # estimate response
        out_tok += "The new name is so much better because it is amazing now and so cool wow yes very beautiful and pretty."
        out_tok += cell_src
    return in_tok, out_tok

in_tok, out_tok = estimate_tokens()

utils.gpt_35_turbo_token_dollar_cost(in_tok, out_tok)

0.470047

In [12]:
# # remove unused functions using vulture (GPT)
# import openai
# openai.api_key = my_key

# # GPT
# def remove_unused(cell_src, function_names):
#     while True:
#         try:
#             completion = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo",
#                 temperature=0,
#                 messages = [
#                 {"role": "user", "content": task2_prompt},
#                 {"role": "user", "content": task2_ex1_input},
#                 {"role": "assistant", "content": task2_ex1_output},
#                 {"role": "user", "content": task2_ex2_input},
#                 {"role": "assistant", "content": task2_ex2_output},
#                 {"role": "user", "content": task2_ex3_input},
#                 {"role": "assistant", "content": task2_ex3_output},
#                 {"role": "user", "content": task2_ex4_input},
#                 {"role": "assistant", "content": task2_ex4_output},
#                 {"role": "user", "content": f"Code:\n```python\n{cell_src}\n```\n\nFunctions to remove:\n{function_names}"}
#             ]
#             )
#         except Exception as e:
#             if 'maximum context length' in str(e):
#                 print('...Error.. too long...' + str(e))
#                 return 'length', None
#             else:
#                 print('...Error.. trying again...' + str(e))
#         else:
#             break
#     return completion.choices[0].finish_reason, completion.choices[0].message["content"]

# gpt_results_code = []
# for i, cell_src in enumerate(random_cells):
#     print(f'Processing file {i}')
#     if gpt_unused_names[i] is None:
#         finish_reason = 'skipped'
#         result = None
#         print('...skipping due to failed identified...')
#     else:
#         finish_reason, result = remove_unused(cell_src, gpt_unused_names[i])
#     print(f'File {i} - {finish_reason}')
#     gpt_results_code.append({'reason': finish_reason, 'result': result})

# # save the results to a file
# with open(GPT_SAVED_FILE_NAME + "_code", 'w') as f:
#     f.write(str(gpt_results_code))

In [13]:
# read in gpt result from file
with open(GPT_SAVED_FILE_NAME + "_code", 'r') as f:
    gpt_results_code = eval(f.read())

In [14]:
# save the updated code to files
gpt_code = []

# get all the code from the results
for i, result in enumerate(gpt_results_code):
    if result['reason'] == 'stop':
        new = result['result'].split("```")
        if len(new) == 1:
            new = None
        else:
            new = new[1]
            if new.startswith('python'):
                new = new[6:].strip("\n")
        gpt_code.append(new)
    else:
        gpt_code.append(None)

In [15]:
# save the updated code to files
import os
if not os.path.exists('gpt_code'):
    os.makedirs('gpt_code')
for i, code in enumerate(gpt_code):
    with open(f'gpt_code/{i}.py', 'w') as f:
        if gpt_unused_names[i] is None or gpt_code[i] is None:
            f.write(random_cells[i])
        else:
            f.write(code)

In [16]:
before = get_unused_data(NUM_FILES, SAMPLES_FOLDER_NAME, 'function')

total_before = sum(len(item) for item in before)
print(f'Total before: {total_before}')

Total before: 302


In [17]:
after = get_unused_data(NUM_FILES, 'gpt_code', 'function')

total_after = sum(len(item) for item in after)
print(f'Total after: {total_after}')

Total after: 20


In [18]:
# List percentage difference between before and after for total
print(f'Total percentage difference: {(total_after - total_before) / total_before * 100}%')

Total percentage difference: -93.37748344370861%


In [19]:
stats_results_unused(gpt_unused_names, before)

GPT before count: 357
Vulture before count: 302
------------
True positives: 289
False positives: 68
False negatives: 12
------------
Files with at least one false positive (and no false negatives)
12: 2 false positives
17: 4 false positives
18: 2 false positives
24: 1 false positives
29: 1 false positives
30: 1 false positives
32: 3 false positives
38: 1 false positives
41: 4 false positives
42: 1 false positives
53: 1 false positives
67: 4 false positives
69: 2 false positives
71: 2 false positives
74: 1 false positives
75: 1 false positives
76: 4 false positives
81: 1 false positives
82: 1 false positives
83: 1 false positives
88: 3 false positives
92: 1 false positives
103: 1 false positives
124: 1 false positives
142: 1 false positives
158: 7 false positives
165: 1 false positives
186: 1 false positives
188: 4 false positives
193: 1 false positives
194: 2 false positives
207: 2 false positives
------------
Files with at least one false negative (and no false positives)
77: 1 false

In [48]:
print(random_cells[85])

def daterange(d1, d2):
    return (d1 + dt.timedelta(days=i) for i in range((d2 - d1).days + 1))

def get_trace_sum_balances(bankdata):
    sum_balances = []
    for bank in bankdata[CURRENT]:
        dates = bankdata[CURRENT][bank]['date']
        balances = bankdata[CURRENT][bank]['balance']
        sum_account = {}
        for date, balance in zip(dates, balances):
            sum_account[date] = balance
                
        sum_balances.append(sum_account)
    
    total = {}
    (ini, fin) = getIntervalDates(bankdata)
    last = 0
    max_amount = 0
    for b in sum_balances:
        for d in daterange(ini, fin):
            if d in b:
                last = b[d]                    
                if d in total:
                    total[d] += b[d]
                    if total[d] > max_amount:
                        max_amount = total[d]
                else:
                    total[d] = b[d]
            else:
                if d in total:
                    total[d] += 

In [47]:
print(gpt_code[85])

class Preprocessor:
    
    def __init__(self, train_data_file, train_label_file, train_ids_file,
                 instr_file, test_data_file=None, test_ids_file=None):
        """A class to process and reformat data
        for use in learning models"""
        
        # initialize the data the data filenames
        self.train_data_file = train_data_file
        self.train_label_file = train_label_file
        self.train_ids_file = train_ids_file
        self.instr_file = instr_file
        
        # test data is optional
        self.test_data_file = test_data_file
        self.test_ids_file = test_ids_file
        
    def read_data(self):
        """Reads in data from the files passed to constructor"""
        
        # read in the data
        train_X_df = pd.read_csv(self.train_data_file)
        train_y_df = pd.read_csv(self.train_label_file)
        train_ids_df = pd.read_csv(self.train_ids_file)
        self.instr_df = pd.read_csv(self.instr_file)
        
        self.fe

In [43]:
before[85]

['read_data', 'process', 'write_data', 'select_features']

In [44]:
after[85]

['read_data', 'process', 'write_data']

In [45]:
gpt_unused_names[85]

['select_features']

In [20]:
# delete 'gpt_code' folder
!rm -rf gpt_code